In [ ]:
!pip install torch
!pip install pandas
!pip install datasets
!pip install transformers
!pip install scikit-learn
!pip install peft
!pip install codecarbon
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import Features, Value, ClassLabel, Dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainerCallback
import time
import matplotlib.pyplot as plt
from codecarbon import EmissionsTracker
import os

In [ ]:
!wget https://perso.univ-lemans.fr/~ndugue/ohsumed-all.zip
!unzip ohsumed-all.zip

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: ohsumed-all/C22/0048429  
  inflating: ohsumed-all/C22/0027535  
  inflating: ohsumed-all/C22/0042334  
  inflating: ohsumed-all/C22/0050186  
   creating: ohsumed-all/C10/
  inflating: ohsumed-all/C10/0042911  
  inflating: ohsumed-all/C10/0035526  
  inflating: ohsumed-all/C10/0031714  
  inflating: ohsumed-all/C10/0019603  
  inflating: ohsumed-all/C10/0047085  
  inflating: ohsumed-all/C10/0042098  
  inflating: ohsumed-all/C10/0013080  
  inflating: ohsumed-all/C10/0035185  
  inflating: ohsumed-all/C10/0025954  
  inflating: ohsumed-all/C10/0029665  
  inflating: ohsumed-all/C10/0019773  
  inflating: ohsumed-all/C10/0030176  
  inflating: ohsumed-all/C10/0003046  
  inflating: ohsumed-all/C10/0012475  
  inflating: ohsumed-all/C10/0029121  
  inflating: ohsumed-all/C10/0009181  
  inflating: ohsumed-all/C10/0018362  
  inflating: ohsumed-all/C10/0021175  
  inflating: ohsumed-all/C10/00315

In [ ]:
labels = []
ids = []
paths = []
fichiers = []

ohsumed_dir = "/content/ohsumed-all"

for category in os.listdir(ohsumed_dir):
    cat_dir = os.path.join(ohsumed_dir, category)
    for fichier in os.listdir(cat_dir):
         if ".ipynb" not in fichier:
            fichier_path = os.path.join(cat_dir, fichier)
            labels.append(category)
            paths.append(fichier_path)
            ids.append(fichier)
            with open(fichier_path) as c_file:
                fichiers.append(c_file.read())

features = Features(
{'text': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=23, id=None, names=list(set(labels))),
 'id': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
})
dico_data = {"text" : fichiers, "label" : labels, "id": ids, "path": paths}
ds = Dataset.from_dict(dico_data, features=features)
ds[0]

ds_split = ds.train_test_split(test_size=0.2, stratify_by_column="label")
ds_split
train_for_ft = ds_split["train"]
test_for_ft = ds_split["test"]

In [ ]:
print(train_for_ft.features["label"].num_classes)

23


In [ ]:
# Préparer le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Charger le modèle pré-entraîné sans fine-tuning
base_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=23)

def tokenize_and_chunk(doc):
    return tokenizer(doc["text"], truncation = True, max_length=512, padding="max_length")

tokenized_train_for_ft = train_for_ft.map(tokenize_and_chunk,batched=True, num_proc=8)
tokenized_test_for_ft = test_for_ft.map(tokenize_and_chunk,batched=True, num_proc=8)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map (num_proc=8):   0%|          | 0/45587 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/11397 [00:00<?, ? examples/s]

In [ ]:
# Récupérer les noms des classes directement à partir du dataset
class_names = tokenized_train_for_ft.features['label'].names

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None, zero_division=1)

    # Calcul des métriques globales (moyenne des classes)
    precision_global = precision.mean()
    recall_global = recall.mean()
    f1_macro = f1.mean()  # Ajout du F1-score macro

    metrics = {
        "accuracy": accuracy,
        "precision_global": precision_global,
        "recall_global": recall_global,
        "f1_macro": f1_macro
    }

    # Ajouter les métriques par classe avec les noms explicites
    for i, (p, r, f) in enumerate(zip(precision, recall, f1)):
        metrics[f"precision_{class_names[i]}"] = p
        metrics[f"recall_{class_names[i]}"] = r
        metrics[f"f1_{class_names[i]}"] = f

    return metrics

# Configurer LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Classification de séquence
    r=8,  # Rang de la décomposition
    lora_alpha=16,  # Facteur d'adaptation
    lora_dropout=0.1,  # Dropout pour LoRA
    target_modules=["q_lin", "v_lin"]  # Modules spécifiques à LoRA dans les transformers
)

# Appliquer LoRA au modèle
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# Configurer l'entraînement avec fine-tuning LoRA
training_args = TrainingArguments(
    output_dir="./results_lora",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    logging_strategy="steps",
    logging_steps=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    overwrite_output_dir=True,
    disable_tqdm=False,
    fp16=True,
    seed=42,
    report_to="wandb"
)

# Initialisation du Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_for_ft ,  # Dataset d'entraînement
    eval_dataset=tokenized_test_for_ft,  # Dataset de test
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

tracker = EmissionsTracker()
tracker.start()

train_result = trainer.train()

emissions = tracker.stop()


trainable params: 755,735 || all params: 67,726,894 || trainable%: 1.1159


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-d510eb8bb8c3>:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
[codecarbon INFO @ 09:07:18] [setup] RAM Tracking...
[codecarbon INFO @ 09:07:18] [setup] CPU Tracking...
[codecarbon WARNING @ 09:07:18] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecar

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:[codecarbon INFO @ 09:07:34] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 09:07:34] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:07:34] Energy consumed for all GPUs : 0.000109 kWh. Total GPU Power : 26.1440120048731 W
[codecarbon INFO @ 09:07:34] 0.000306 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:07:49] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 09:07:49] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:07:49] Energy consumed for all GPUs : 0.000218 kWh. Total GPU Power : 26.175215882269192 W
[codecarbon INFO @ 09:07:49] 0.000612 kWh of elec

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xakaa (xakaa-le-mans-universit-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision Global,Recall Global,F1 Macro,Precision C05,Recall C05,F1 C05,Precision C13,Recall C13,F1 C13,Precision C23,Recall C23,F1 C23,Precision C12,Recall C12,F1 C12,Precision C11,Recall C11,F1 C11,Precision C10,Recall C10,F1 C10,Precision C17,Recall C17,F1 C17,Precision C09,Recall C09,F1 C09,Precision C20,Recall C20,F1 C20,Precision C07,Recall C07,F1 C07,Precision C18,Recall C18,F1 C18,Precision C01,Recall C01,F1 C01,Precision C19,Recall C19,F1 C19,Precision C14,Recall C14,F1 C14,Precision C06,Recall C06,F1 C06,Precision C16,Recall C16,F1 C16,Precision C04,Recall C04,F1 C04,Precision C22,Recall C22,F1 C22,Precision C15,Recall C15,F1 C15,Precision C08,Recall C08,F1 C08,Precision C02,Recall C02,F1 C02,Precision C03,Recall C03,F1 C03,Precision C21,Recall C21,F1 C21
1,1.816500,1.749633,0.439677,0.445647,0.383251,0.374107,0.359756,0.351190,0.355422,0.350093,0.578462,0.436195,0.279290,0.122789,0.170582,0.466019,0.571429,0.513369,0.468750,0.525000,0.495283,0.419463,0.487013,0.450721,0.327078,0.377709,0.350575,0.362963,0.342657,0.352518,0.384120,0.574639,0.460450,0.421053,0.228571,0.296296,0.463104,0.473958,0.468468,0.429648,0.336614,0.377483,0.391892,0.167630,0.234818,0.548107,0.723770,0.623808,0.435277,0.511706,0.470407,0.262411,0.170507,0.206704,0.507523,0.692733,0.585838,1.000000,0.000000,0.000000,0.418919,0.121569,0.188450,0.406491,0.507722,0.451502,0.301587,0.081197,0.127946,0.692308,0.317647,0.435484,0.554031,0.550256,0.552137
2,1.703500,1.682825,0.450469,0.407345,0.402997,0.395525,0.368421,0.333333,0.350000,0.370874,0.587692,0.454762,0.290657,0.131113,0.180710,0.472930,0.589286,0.524735,0.472477,0.515000,0.492823,0.459951,0.492208,0.475533,0.383607,0.362229,0.372611,0.349315,0.356643,0.352941,0.432873,0.553772,0.485915,0.364865,0.257143,0.301676,0.477500,0.497396,0.487245,0.418349,0.448819,0.433048,0.355769,0.213873,0.267148,0.526919,0.754098,0.620364,0.449782,0.516722,0.480934,0.333333,0.165899,0.221538,0.509849,0.695103,0.588235,0.000000,0.000000,0.000000,0.365672,0.192157,0.251928,0.424581,0.440154,0.432227,0.285714,0.188034,0.226804,0.677966,0.470588,0.555556,0.577519,0.507666,0.540345
3,1.475700,1.658724,0.453102,0.414894,0.407234,0.402867,0.367601,0.351190,0.359209,0.397089,0.587692,0.473945,0.285593,0.175338,0.217279,0.477589,0.613095,0.536924,0.481651,0.525000,0.502392,0.474576,0.472727,0.473650,0.384615,0.386997,0.385802,0.342466,0.349650,0.346021,0.427518,0.558587,0.484342,0.353659,0.276190,0.310160,0.503979,0.494792,0.499343,0.430528,0.433071,0.431796,0.351852,0.219653,0.270463,0.569814,0.702459,0.629222,0.453488,0.521739,0.485226,0.382550,0.262673,0.311475,0.518792,0.665087,0.582901,0.000000,0.000000,0.000000,0.377953,0.188235,0.251309,0.406149,0.484556,0.441901,0.277778,0.149573,0.194444,0.711538,0.435294,0.540146,0.565789,0.512777,0.537980


[codecarbon INFO @ 09:08:19] Energy consumed for RAM : 0.000079 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 09:08:19] Energy consumed for all CPUs : 0.000708 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:08:19] Energy consumed for all GPUs : 0.000471 kWh. Total GPU Power : 34.26321150583048 W
[codecarbon INFO @ 09:08:19] 0.001259 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:08:34] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 09:08:34] Energy consumed for all CPUs : 0.000885 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:08:34] Energy consumed for all GPUs : 0.000754 kWh. Total GPU Power : 67.96726294303001 W
[codecarbon INFO @ 09:08:34] 0.001738 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:08:49] Energy consumed for RAM : 0.000119 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 09:08:49] Energy consumed for all CPUs : 0.001062 kWh. Total CPU Power : 42.5 W
[codeca

In [ ]:
import torch
import pandas as pd
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from codecarbon import EmissionsTracker

# ✅ CHOIX : Sélectionner les paramètres à entraîner
freeze_mode = "last_layers"  # "head", "last_layers", "embeddings", "none"

if freeze_mode == "head":
    for param in base_model.distilbert.parameters():
        param.requires_grad = False  # Gèle tout sauf la tête

elif freeze_mode == "last_layers":
    for param in base_model.distilbert.parameters():
        param.requires_grad = False  # Gèle tout
    for layer in base_model.distilbert.transformer.layer[-2:]:  # Dégeler les 2 dernières couches
        for param in layer.parameters():
            param.requires_grad = True

elif freeze_mode == "embeddings":
    for param in base_model.distilbert.embeddings.parameters():
        param.requires_grad = False  # Gèle uniquement les embeddings

elif freeze_mode == "none":
    pass  # Fine-tune complet

# Vérifier les paramètres entraînables
for name, param in base_model.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")

# Fonction de calcul des métriques
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average=None, zero_division=1)

    return {
        "accuracy": accuracy,
        "precision_global": precision.mean(),
        "recall_global": recall.mean(),
        "f1_macro": f1.mean()
    }

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results_sans_lora",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    logging_strategy="steps",
    logging_steps=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    overwrite_output_dir=True,
    disable_tqdm=False,
    fp16=True,
    seed=42,
    report_to="wandb"
)

# Initialisation du Trainer
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_train_for_ft,
    eval_dataset=tokenized_test_for_ft,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Suivi de l'empreinte carbone
tracker = EmissionsTracker()
tracker.start()

# Entraînement
train_result = trainer.train()

# Fin du suivi carbone
emissions = tracker.stop()


distilbert.embeddings.word_embeddings.weight: requires_grad = False
distilbert.embeddings.position_embeddings.weight: requires_grad = False
distilbert.embeddings.LayerNorm.weight: requires_grad = False
distilbert.embeddings.LayerNorm.bias: requires_grad = False
distilbert.transformer.layer.0.attention.q_lin.weight: requires_grad = False
distilbert.transformer.layer.0.attention.q_lin.bias: requires_grad = False
distilbert.transformer.layer.0.attention.k_lin.weight: requires_grad = False
distilbert.transformer.layer.0.attention.k_lin.bias: requires_grad = False
distilbert.transformer.layer.0.attention.v_lin.weight: requires_grad = False
distilbert.transformer.layer.0.attention.v_lin.bias: requires_grad = False
distilbert.transformer.layer.0.attention.out_lin.weight: requires_grad = False
distilbert.transformer.layer.0.attention.out_lin.bias: requires_grad = False
distilbert.transformer.layer.0.sa_layer_norm.weight: requires_grad = False
distilbert.transformer.layer.0.sa_layer_norm.bias: 

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-ae6d1259a9aa>:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
[codecarbon INFO @ 12:16:38] [setup] RAM Tracking...
[codecarbon INFO @ 12:16:38] [setup] CPU Tracking...
[codecarbon WARNING @ 12:16:38] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 12:16:39] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 12:16:39] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 12:16:39] [setup] GPU Tracking...


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:[codecarbon INFO @ 12:16:54] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:16:54] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:16:54] Energy consumed for all GPUs : 0.000120 kWh. Total GPU Power : 28.74454086613241 W
[codecarbon INFO @ 12:16:54] 0.000317 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:17:09] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:17:09] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:17:09] Energy consumed for all GPUs : 0.000240 kWh. Total GPU Power : 28.778633039275274 W
[codecarbon INFO @ 12:17:09] 0.000633 kWh of ele

Epoch,Training Loss,Validation Loss,Accuracy,Precision Global,Recall Global,F1 Macro
1,1.560800,1.622662,0.460999,0.438153,0.414347,0.410789


[codecarbon INFO @ 12:17:24] Energy consumed for RAM : 0.000059 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:17:24] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:17:24] Energy consumed for all GPUs : 0.000421 kWh. Total GPU Power : 43.45097579654778 W
[codecarbon INFO @ 12:17:24] 0.001011 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:17:39] Energy consumed for RAM : 0.000079 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:17:39] Energy consumed for all CPUs : 0.000708 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:17:39] Energy consumed for all GPUs : 0.000707 kWh. Total GPU Power : 68.80233071568587 W
[codecarbon INFO @ 12:17:39] 0.001495 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:17:54] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:17:54] Energy consumed for all CPUs : 0.000885 kWh. Total CPU Power : 42.5 W
[codeca

Epoch,Training Loss,Validation Loss,Accuracy,Precision Global,Recall Global,F1 Macro
1,1.560800,1.622662,0.460999,0.438153,0.414347,0.410789
2,1.652500,1.527161,0.469685,0.446748,0.417216,0.421202
3,1.299800,1.505443,0.470211,0.434772,0.450655,0.436693


[codecarbon INFO @ 12:26:54] Energy consumed for RAM : 0.000811 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:26:54] Energy consumed for all CPUs : 0.007258 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:26:54] Energy consumed for all GPUs : 0.011304 kWh. Total GPU Power : 69.10263677945031 W
[codecarbon INFO @ 12:26:54] 0.019373 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:27:09] Energy consumed for RAM : 0.000831 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:27:09] Energy consumed for all CPUs : 0.007435 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:27:09] Energy consumed for all GPUs : 0.011592 kWh. Total GPU Power : 69.0234235226717 W
[codecarbon INFO @ 12:27:09] 0.019858 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:27:24] Energy consumed for RAM : 0.000851 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:27:24] Energy consumed for all CPUs : 0.007611 kWh. Total CPU Power : 42.5 W
[codecar

In [ ]:
import pandas as pd

def generate_metrics_report(trainer, final_metrics, training_args, emissions, tokenizer=None, lora_config=None):
    """
    Génère un rapport compact avec les métriques clés.
    Affiche automatiquement les infos liées à LoRA si `lora_config` est fourni.
    """
    def safe_format(value):
        return f"{value:.4f}" if isinstance(value, (int, float)) else str(value)

    history = pd.DataFrame(trainer.state.log_history)

    # Calcul du temps moyen par époque
    if 'epoch' in history.columns and 'train_runtime' in history.columns:
        epoch_times = history[history['epoch'].notna()].groupby('epoch')['train_runtime'].mean()
        mean_epoch_time = epoch_times.mean() if not epoch_times.empty else "N/A"
    else:
        mean_epoch_time = "N/A"

    # Section LoRA (optionnelle)
    lora_section = (
        f"- **LoRA Config** : r={lora_config.r}, alpha={lora_config.lora_alpha}, dropout={lora_config.lora_dropout}"
        if lora_config else "- **LoRA** : non utilisé"
    )

    # Choix du nom de fichier / courbe
    suffix = "_lora" if lora_config else "_sans_lora"
    report_path = f"./results{suffix}/training_report{suffix}.txt"
    learning_curve_path = f"./results{suffix}/learning_curves{suffix}.png"

    report = f"""
# 📊 Rapport d'entraînement et d'évaluation

## 🔍 **Résultats**
- **Accuracy** : {safe_format(final_metrics.get('eval_accuracy', 'N/A'))}
- **F1-score (macro)** : {safe_format(final_metrics.get('eval_f1_macro', 'N/A'))}
- **Precision globale** : {safe_format(final_metrics.get('eval_precision_global', 'N/A'))}
- **Recall global** : {safe_format(final_metrics.get('eval_recall_global', 'N/A'))}
- **Loss finale** : {safe_format(final_metrics.get('eval_loss', 'N/A'))}

## ⚙️ **Hyperparamètres**
- **Epochs** : {training_args.num_train_epochs}
- **Batch (train/eval)** : {training_args.per_device_train_batch_size} / {training_args.per_device_eval_batch_size}
- **Learning Rate** : {training_args.learning_rate}
- **Weight Decay** : {training_args.weight_decay}
{lora_section}

## 🧠 **Tokenizer**
- **Tokenizer utilisé** : {getattr(tokenizer, 'name_or_path', 'Non spécifié') if tokenizer else 'Non fourni'}

## ⏱ **Temps d'entraînement**
- **Temps moyen par époque** : {safe_format(mean_epoch_time)} sec

## 🌱 **Empreinte carbone**
- **CO₂ estimé** : {safe_format(emissions)} kg

## 📈 **Courbes d'apprentissage**
📌 `{learning_curve_path}`
"""
    with open(report_path, "w") as f:
        f.write(report)
    print(report)

In [ ]:
final_metrics = trainer.evaluate()
generate_metrics_report(trainer, final_metrics, training_args, emissions, tokenizer)
print(final_metrics)


# 📊 Rapport d'entraînement et d'évaluation

## 🔍 **Résultats**
- **Accuracy** : 0.4702
- **F1-score (macro)** : 0.4367
- **Precision globale** : 0.4348
- **Recall global** : 0.4507
- **Loss finale** : 1.5054

## ⚙️ **Hyperparamètres**
- **Epochs** : 3
- **Batch (train/eval)** : 8 / 8
- **Learning Rate** : 5e-05
- **Weight Decay** : 0.01
- **LoRA** : non utilisé

## 🧠 **Tokenizer**
- **Tokenizer utilisé** : distilbert-base-uncased

## ⏱ **Temps d'entraînement**
- **Temps moyen par époque** : 1229.2075 sec

## 🌱 **Empreinte carbone**
- **CO₂ estimé** : 0.0184 kg

## 📈 **Courbes d'apprentissage**
📌 `./results_sans_lora/learning_curves_sans_lora.png`

{'eval_loss': 1.5054434537887573, 'eval_accuracy': 0.4702114591559182, 'eval_precision_global': 0.4347716134163395, 'eval_recall_global': 0.45065452423515995, 'eval_f1_macro': 0.43669304012458393, 'eval_runtime': 58.7162, 'eval_samples_per_second': 194.103, 'eval_steps_per_second': 24.269, 'epoch': 3.0}


In [ ]:
!zip -r mon_dossier.zip ./

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  adding: ohsumed-all/C16/0000239 (deflated 47%)
  adding: ohsumed-all/C16/0041616 (deflated 40%)
  adding: ohsumed-all/C16/0032476 (deflated 54%)
  adding: ohsumed-all/C16/0000847 (deflated 48%)
  adding: ohsumed-all/C16/0000268 (deflated 50%)
  adding: ohsumed-all/C16/0033516 (deflated 54%)
  adding: ohsumed-all/C16/0022059 (deflated 51%)
  adding: ohsumed-all/C16/0009138 (deflated 50%)
  adding: ohsumed-all/C16/0014466 (deflated 43%)
  adding: ohsumed-all/C16/0009025 (deflated 50%)
  adding: ohsumed-all/C16/0015528 (deflated 57%)
  adding: ohsumed-all/C16/0041512 (deflated 52%)
  adding: ohsumed-all/C16/0031302 (deflated 44%)
  adding: ohsumed-all/C16/0012810 (deflated 51%)
  adding: ohsumed-all/C16/0020671 (deflated 45%)
  adding: ohsumed-all/C16/0026100 (deflated 50%)
  adding: ohsumed-all/C16/0006857 (deflated 51%)
  adding: ohsumed-all/C16/0029788 (deflated 52%)
  adding: ohsumed-all/C16/0019927 (defla

In [ ]:
from google.colab import files
files.download("mon_dossier.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>